In [12]:

from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Pt
import pandas as pd

import sys
# adding UtilityFuncts to the system path
sys.path.insert(0, '/notebooks/Utility')
import UtilityFuncts as uf

In [13]:
import os

In [14]:
# hf_token = os.environ.get('secret:hf_token')

In [15]:
# print(hf_token)